## Стековый компилятор формул (ipynb-версия)

Для автоматической проверки кода с помощью `pycodestyle` можно использовать следующие «магические команды»:

In [4]:
# При необходимости: pip install flake8 pycodestyle_magic
%load_ext pycodestyle_magic
%pycodestyle_on

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [5]:
class Stack:
    def __init__(self):
        self.array = []

    def push(self, c):
        self.array.append(c)

    def pop(self):
        return self.array.pop()

    def top(self):
        return self.array[len(self.array) - 1]

Стековый компилятор формул преобразует правильные
арифметические формулы (цепочки языка, задаваемого
грамматикой `G0`) в программы для стекового калькулятора
(цепочки языка, определяемого грамматикой `Gs`):

    G0:
        F  ->  T  |  F+T  |  F-T
        T  ->  M  |  T*M  |  T/M
        M  -> (F) |   V
        V  ->  a  |   b   |   c   |  ...  |    z

    Gs:
        e  ->  e e + | e e - | e e * | e e / |
                     | a | b | ... | z

В качестве операндов в формулах допустимы только
однобуквенные имена переменных `[a-z]`.

In [6]:
import re


class Compf:

    SYMBOLS = re.compile("[a-z]")

    def __init__(self):
        self.s = Stack()
        self.data = []

    def compile(self, str):
        self.data.clear()
        for c in "(" + str + ")":
            self.process_symbol(c)
        return " ".join(self.data)

    def process_symbol(self, c):
        if c == "(":
            self.s.push(c)
        elif c == ")":
            self.process_suspended_operators(c)
            self.s.pop()
        elif c in "+-*/":
            self.process_suspended_operators(c)
            self.s.push(c)
        else:
            self.check_symbol(c)
            self.process_value(c)

    def process_suspended_operators(self, c):
        while self.is_precedes(self.s.top(), c):
            self.process_oper(self.s.pop())

    def process_value(self, c):
        self.data.append(c)

    def process_oper(self, c):
        self.data.append(c)

    @classmethod
    def check_symbol(self, c):
        if not self.SYMBOLS.match(c):
            raise Exception(f"Недопустимый символ '{c}'")

    @staticmethod
    def priority(c):
        return 1 if (c == "+" or c == "-") else 2

    @staticmethod
    def is_precedes(a, b):
        if a == "(":
            return False
        elif b == ")":
            return True
        else:
            return Compf.priority(a) >= Compf.priority(b)

Тесты тоже можно использовать в Jupyter Notebook:

In [7]:
import unittest


class TestCompf(unittest.TestCase):

    def setUp(self):
        self.c = Compf()

    def test_one_symbol(self):
        self.assertEqual(self.c.compile("a"), "a")

    def test_correct_operations1(self):
        self.assertEqual(self.c.compile("a+b"), "a b +")

    def test_correct_operations2(self):
        self.assertEqual(self.c.compile("a-b"), "a b -")

    def test_correct_operations3(self):
        self.assertEqual(self.c.compile("a*b"), "a b *")

    def test_correct_operations4(self):
        self.assertEqual(self.c.compile("a/b"), "a b /")

    def test_operations_order1(self):
        self.assertEqual(self.c.compile("a+c*b"), "a c b * +")

    def test_operations_order2(self):
        self.assertEqual(self.c.compile("a*b/c"), "a b * c /")

    def test_operations_order3(self):
        self.assertEqual(self.c.compile("a*(b/c)"), "a b c / *")

    def test_parentheses1(self):
        self.assertEqual(self.c.compile("(a)"), "a")

    def test_parentheses2(self):
        self.assertEqual(self.c.compile("(((((a))))"), "a")

    def test_parentheses3(self):
        self.assertEqual(self.c.compile("(((((a+b))))"), "a b +")

    def test_parentheses4(self):
        self.assertEqual(self.c.compile("(((((((a+b)*((a+b)))))))"),
                         "a b + a b + *")

    def test_expressions1(self):
        self.assertEqual(self.c.compile("(a+b)*c+(d-e)/f"),
                         "a b + c * d e - f / +")

    def test_expressions2(self):
        self.assertEqual(self.c.compile("c*(c+c+c+c/(c-c-c-c))"),
                         "c c c + c + c c c - c - c - / + *")

    def test_expressions3(self):
        self.assertEqual(self.c.compile("a/b*c+d*e/(f+g)"),
                         "a b / c * d e * f g + / +")

    def test_expressions4(self):
        self.assertEqual(self.c.compile("a/b*(c+d*e)/(f+g)"),
                         "a b / c d e * + * f g + /")

    def test_expressions5(self):
        self.assertEqual(self.c.compile("a+b*(c-d)*(c+(d-e)/a)/a"),
                         "a b c d - * c d e - a / + * a / +")

    def test_expressions6(self):
        self.assertEqual(self.c.compile("((c+(c*(c+(c+c/c)))))/(c+c)"),
                         "c c c c c c / + + * + c c + /")

    def test_expressions7(self):
        self.assertEqual(self.c.compile("c+(c+(c*(c+(c/(c*(c+c))))))"),
                         "c c c c c c c c + * / + * + +")


unittest.main(argv=[''], exit=False)

...................
----------------------------------------------------------------------
Ran 19 tests in 0.006s

OK


Запуск компилятора:

In [8]:
c = Compf()
while True:
    s = input("Арифметическая  формула: ")
    if len(s) == 0:
        break
    print(f"Результат её компиляции: {c.compile(s)}\n")
print("Завершение работы")

Результат её компиляции: a b +

Завершение работы


### **Модификация проекта**

---

*Постановка задачи:*
> Формулы, содержащие только записанные в шестнадцатеричной системе натуральные числа (запись которых обязана начинаться с 0x или 0X), абсолютная величина которых не превосходит 3999, компилируются в программы для стекового калькулятора, содержащие римские числа (в записи которых используются цифры I, V, X, L, C, D и M).  
  
*Идея решения:*  
  
Т.к. в проекте **уже** реализован стековый компилятор для арифмитических выражений с буквенными переменными, нет смысла *"изобретать велосипед"* или же редактировать уже имеющийся код, интегрируя в него решение нашей задачи, и достаточно заменить в выражении все 16-ричные числа на буквенные переменные, а после компиляции сделать обратную замену, попутно переведя 16-ричные числа в римские.  

В случае необходимости, разумеется, можно без труда реализовать полный функционал обеих частей проекта(основной и модификации) в рамках одного запуска компилятора, но это не было явно указано при постановке задачи, посему мною было принято решение для большей понятности кода, все-таки реализованную на базе проекта модификацию вынести в отдельный пусковой блок кода.

Стековый компилятор формул преобразует правильные
арифметические формулы (цепочки языка, задаваемого
грамматикой `G0`) в программы для стекового калькулятора
(цепочки языка, определяемого грамматикой `Gs`):

    G0:
        F  ->  T  |  F+T  |  F-T
        T  ->  M  |  T*M  |  T/M
        M  -> (F) |   V
        V  -> 0xK |  0XK
        K  ->  1K |  2K   |  ...  |   9K   |   aK   |  ...  |   fK   |
                  |   1   |  ...  |   9    |   a    |  ...  |   f   |

    Gs:
        e  ->  e e + | e e - | e e * | e e / |
                     | Ie | Ve | Xe | Le | Ce | De | Me |
                     | I  | V  | X  | L  | C  | D  | M  |


Функция перевода 16-ричных чисел в римские:

In [15]:
def hex_to_roman(hex):
    dec = int(hex, 16)
    val = [
        1000, 900, 500, 400,
        100, 90, 50, 40,
        10, 9, 5, 4,
        1
        ]
    syb = [
        "M", "CM", "D", "CD",
        "C", "XC", "L", "XL",
        "X", "IX", "V", "IV",
        "I"
        ]
    rom = ''
    i = 0
    while dec > 0:
        for _ in range(dec // val[i]):
            rom += syb[i]
            dec -= val[i]
        i += 1
    return rom

Функция для замены всех 16-ричных чисел на буквы:

In [20]:
def hex2let(expression):
    # Регулярное выражение для поиска
    # шестнадцатеричных чисел, начинающихся с 0x или 0X
    pattern = r'0[xX][0-9A-Fa-f]+'

    # Находим все шестнадцатеричные числа в строке
    hex_num = re.findall(pattern, expression)

    # Словарь для замены чисел на переменные
    replacements = {}
    SYMBOLS = 'abcdefghijklmnopqrstuvwxyz'

    # Заменяем шестнадцатеричные числа на буквы
    for i, hex in enumerate(hex_num):
        replacements[hex] = SYMBOLS[i]

    # Функция для замены
    def replace(match):
        return replacements[match.group(0)]

    # Заменяем числа в выражении
    modified_expression = re.sub(pattern, replace, expression)

    # Переводим 16-ричные числа в римские
    rom_num = list(map(hex_to_roman, hex_num))

    return modified_expression, rom_num

Функция для замены переменных для компиляции на римские числа:

In [21]:
def let2rom(expression, rom_num):
    # Регулярное выражение для поиска латинских букв в выражении
    SYMBOLS = re.compile("[a-z]")

    # Находим все буквы в строке
    letters = re.findall(SYMBOLS, expression)

    # Словарь для замены букв на римские числа
    replacements = {letter: rom_num[i] for i, letter in enumerate(letters)}

    # Функция для замены
    def replace(match):
        return replacements[match.group(0)]

    # Заменяем буквы в выражении
    modified_expression = re.sub(SYMBOLS, replace, expression)

    return modified_expression

Тесты:

In [25]:
import unittest


class TestHex2Roman(unittest.TestCase):
    # Блок тестов на перевод 16-ричных чисел в римские
    def test_hex2rom1(self):
        self.assertEqual(hex_to_roman('0x1'), 'I')

    def test_hex2rom2(self):
        self.assertEqual(hex_to_roman('0XA'), 'X')

    def test_hex2rom3(self):
        self.assertEqual(hex_to_roman('0x64'), 'LXIV')

    def test_hex2rom4(self):
        self.assertEqual(hex_to_roman('0X3E8'), 'MMMDCCCLXXXVIII')

    # Блок тестов на замену 16-ричных чисел буквенными переменными
    def test_hex2let1(self):
        self.assertEqual(hex2let('0x1 + 0xA'), ('a + b', ['I', 'X']))

    def test_hex2let2(self):
        self.assertEqual(hex2let('0x64 * 0x3E8'),
                         ('a * b', ['LXIV', 'MMMDCCCLXXXVIII']))

    # Блок тестов на замену буквенных переменных
    # соответстующими римскими числами
    def test_let2rom1(self):
        self.assertEqual(let2rom('a + b', ['I', 'X']), 'I + X')

    def test_let2rom2(self):
        self.assertEqual(let2rom('a * b', ['LXIV', 'MMMDCCCLXXXVIII']),
                         'LXIV * MMMDCCCLXXXVIII')

Запуск компилятора для 16-ричных чисел:

In [13]:
c = Compf()
while True:
    s = input("Арифметическая  формула: ")
    if len(s) == 0:
        break
    s, rom_num = hex2let(s)
    res = let2rom(c.compile(s), rom_num)
    print(f"Результат её компиляции: {res}\n")
print("Завершение работы")

Результат её компиляции: CCXCI CXVIII -

Завершение работы
